# Домашка 5. NN

## DSSM
1. Использовать информацию о качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования. (3 балла):
    * вместо единички добавляем к матрице взаимодействий `watched_pct`
2. Добавить текстовые признаки в модель (3 балла):
    * Для списков режиссеров, стран -- добавил умный one-hot encoding (сплит по запятой)
    * Для опиcаний - ruBERT использовал.
3. Пробить скор бейзлайна DSSM модели 0.075. (2 балла)
   * Побил: 0.086
4. Обернуть в сервис, который должен  критериям читаемости и воспроизводимости. (5 балла)
   * Обернул (и через бота проверку прошел):
   * Класс модели: `dev_models/dssmmodel.py`
   * Класс для чтения в продакшне: `models/DSSM.py`


In [2]:
from collections import Counter

from dssmmodel import DSSMModel

import pickle

import tensorflow.keras.backend as K
from tensorflow import keras
from random import randint

from tqdm import tqdm
import pandas as pd
from rectools.dataset import Dataset
import warnings

from sklearn.metrics.pairwise import euclidean_distances as ED

import numpy as np

from transformers import BertTokenizer, BertModel
import torch

from dev_eval import read_kion_dataset

# pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")
tqdm.pandas()

2023-12-16 15:28:07.353870: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-16 15:28:07.360352: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-16 15:28:07.429880: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-16 15:28:07.430653: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-16 15:28:08.057665: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

# Data

Подгружаем датасет кион используя дополнительную функцию

In [3]:
kion_data = read_kion_dataset(fast_check=1)
interactions = kion_data["interactions"]
users = kion_data["users"]
items = kion_data["items"]

In [5]:
interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


## Препроцессинг фичей айтемов

In [6]:
items.head(3)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."


Работаем 3 способами с фичами:
* просто категориальные -- one hot encoding (как в лекции)
* Раньше (в лекции) директоры и страны были в категориальных фичах, но это не совсем так, так как там через запятую могут стоять несколько категорий (например, `США, Испания`, `США`,  `Испания` были 3 разными классами, хотя это можно уместить в два класса: просто ставить единички больше чем в 1 классе). Это сделаем для `['countries', 'directors', 'genres', 'keywords']`
* заюзаем берт для энкодинга descriptions и keywords

In [19]:
## просто one hot encoding
item_cat_feats = ["content_type", "for_kids", "age_rating", "studios"]
items_ohe_df = items.item_id

for feat in item_cat_feats:
    ohe_feat = pd.get_dummies(items[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat], axis=1)

## one hot encoding для фичей через запятую
comma_cat_feats = ["countries", "directors", "genres"]

for feat in comma_cat_feats:
    ohe_feat = items[feat].str.get_dummies(sep=", ").add_prefix(f"{feat}_")
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat], axis=1)

items_ohe_df.head()

,item_id,content_type_film,content_type_series,for_kids_0.0,for_kids_1.0,age_rating_0.0,age_rating_6.0,age_rating_12.0,age_rating_16.0,age_rating_18.0,...,genres_фильмы,genres_фильмы hbo,genres_фильмы-спектакли,genres_фитнес,genres_футбол,genres_фэнтези,genres_хочу всё знать,genres_шоу,genres_экранизации,genres_юмор
0,10711,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2508,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10716,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,7868,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,16268,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# подгружаем берт
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")


def encode_description(review):
    encoded_review = tokenizer(review, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**encoded_review)
    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings.numpy()[0]


## пришлось отдельно на коллабе запустить берт и сохранить (на компе валилось)
embedding_description = items.description.fillna("").progress_apply(lambda x: encode_description(x))
embeddings = pd.DataFrame([list(i) for i in embedding_description.to_list()])
embeddings.to_csv("embeddings.csv", index=False)

In [20]:
## read
embeddings = pd.read_csv("../data/embeddings.csv").add_prefix(f"emb_")
embeddings
## MERGE with _df
items_ohe_df = pd.concat([items_ohe_df, embeddings], axis=1)
items_ohe_df.head()

,item_id,content_type_film,content_type_series,for_kids_0.0,for_kids_1.0,age_rating_0.0,age_rating_6.0,age_rating_12.0,age_rating_16.0,age_rating_18.0,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
0,10711,1,0,0,0,0,0,0,1,0,...,0.135498,-0.050018,-0.064492,-0.170823,0.039486,0.099307,0.054217,0.106805,-0.135492,-0.241274
1,2508,1,0,0,0,0,0,0,1,0,...,0.071463,0.092410,-0.030258,-0.044130,-0.108914,0.071249,0.086019,0.237594,-0.150463,0.009551
2,10716,1,0,0,0,0,0,0,1,0,...,-0.035445,-0.028603,-0.054121,-0.195034,-0.008158,0.033368,0.035486,0.225539,-0.102949,-0.111427
3,7868,1,0,0,0,0,0,0,1,0,...,0.037582,-0.009220,-0.215980,0.006530,-0.111757,0.168911,0.061653,0.165149,-0.002899,-0.256809
4,16268,1,0,0,0,0,0,1,0,0,...,0.281592,0.026697,0.104811,-0.012988,0.019750,0.234832,0.125230,-0.017858,-0.121802,-0.375776


## Юзеры

Категориальные признаки (все) one-hot encod'им.

In [21]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами
# из датафрейма с взаимодействиями
users_ohe_df = users.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом,
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,sex_Ж,sex_М,kids_flg_0,kids_flg_1
0,973171,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1
1,962099,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
2,1047345,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0
3,721985,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0
4,704055,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0


## Матица взаимодействий

В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил.

Давайте отфильтруем такие события: малоактивных юзеров и непопулярные фильмы.

Тут все, как и на паре.

In [22]:
print(f"N users before: {interactions.df.user_id.nunique()}")
print(f"N items before: {interactions.df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions.df[interactions.df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели
# больше 10 фильмов
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 962179
N items before: 15706

N users after: 79515
N items after: 6901


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [24]:
common_users = set(interactions_df.user_id) & set(users_ohe_df.user_id)
common_items = set(interactions_df.item_id) & set(items_ohe_df.item_id)

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

print(len(interactions_df.item_id.unique()), items_ohe_df.shape[0])
print(len(interactions_df.user_id.unique()), users_ohe_df.shape[0])

6897 6897
65974 65974


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [25]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,datetime,weight,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250.0,72.0,10616,3944
1,699317,1659,2021-05-29,8317.0,100.0,42131,675
6,1016458,354,2021-08-14,1672.0,25.0,61024,139
7,884009,693,2021-08-04,703.0,14.0,53150,279
14,5324,8437,2021-04-18,6598.0,92.0,310,3485


Отнормируем матрицу взаимодействий


__NOTE__ Тут мы выполняем 1 из куска задания: Используем информацию о проценте просмотра айтема, а не просто единичку 

In [26]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), interactions_df.iid.nunique()))

# Используем информацию о качестве (проценте просмотра) взаимодействия юзеров с айтемами
# ЭТО КУСОК ЗАДАНИЯ
for user_id, item_id, weight in zip(interactions_df.uid, interactions_df.iid, interactions_df.watched_pct):
    interactions_vec[user_id, item_id] += weight / 100


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [27]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

6897
6897
65974
65974
set()


Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари

```
{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.
```

In [28]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [29]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

# Обучение

Начинаем с лосса

In [30]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims : n_dims * 2]
    negative = y_pred[:, n_dims * 2 : n_dims * 3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)  # возвращаем ноль, если лосс отрицательный

    return loss

Сделаем простой генератор. Он будет брать рандромного юзера, и два разных айтема - хороший пример и плохой:

хорошим примером будет тот айтем, который был взят из датасета взаимодействий в соответствии с распределением просмотренных айтемов для этого юзера;
а плохим айтемом будет просто любой другой случайный айтем*

In [31]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0] - 1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [
            np.array(uid_meta),
            np.array(uid_interaction),
        ]


# инициализируем генератор
gen = generator(
    items=items_ohe_df.drop(["item_id"], axis=1),
    users=users_ohe_df.drop(["user_id"], axis=1),
    interactions=interactions_vec,
)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 16)
вектор взаимодействий юзера с айтемами: (1024, 6897)
вектор 'хорошего' айтема: (1024, 9841)
вектор 'плохого' айтема: (1024, 9841)

вектор фичей юзера: (1024, 16)
вектор взаимодействий юзера с айтемами: (1024, 6897)


In [32]:
### Модель

In [33]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1],)
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1],)

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1],)

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (9841,)
USER_META_MODEL_SHAPE: (16,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [34]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

    # полносвязный слой
    layer_1 = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(layer_1)

    add = keras.layers.Add()([layer_1, layer_2])

    # выходной слой
    out = keras.layers.Dense(
        N_FACTORS,
        activation="linear",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(add)

    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp_meta)

    layer_1_interaction = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(
        N_FACTORS,
        activation="elu",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(layer_1_meta)

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])

    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])

    # выходной слой
    out = keras.layers.Dense(
        N_FACTORS,
        activation="linear",
        use_bias=False,
        kernel_regularizer=keras.regularizers.l2(1e-6),
        activity_regularizer=keras.regularizers.l2(l2=1e-6),
    )(concat_meta_interaction)

    return keras.models.Model([inp_meta, inp_interaction], out)


# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [35]:
model_name = "recsys_resnet_linear"

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f"runs/{model_name}")

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + "/epoch{epoch}-{loss:.2f}.h5", monitor="loss")

In [36]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [37]:
# модель айтема
item_model().summary()
# модель юзера
user_model().summary()
# общая модель
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 9841)]       0           []                               
                                                                                                  
 dense_7 (Dense)                (None, 128)          1259648     ['input_8[0][0]']                
                                                                                                  
 dense_8 (Dense)                (None, 128)          16384       ['dense_7[0][0]']                
                                                                                                  
 add_2 (Add)                    (None, 128)          0           ['dense_7[0][0]',                
                                                                  'dense_8[0][0]']          

# Train

In [38]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(
    generator(
        items=items_ohe_df.drop(["item_id"], axis=1),
        users=users_ohe_df.drop(["user_id"], axis=1),
        interactions=interactions_vec,
        batch_size=64,
    ),
    steps_per_epoch=100,
    epochs=30,
    initial_epoch=0,
    callbacks=[decay, t_board, check],
)

Epoch 1/30


2023-12-16 15:30:41.584112: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - 32s 315ms/step - loss: 0.5300 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 30s 307ms/step - loss: 0.3407 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 30s 307ms/step - loss: 0.3104 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 31s 318ms/step - loss: 0.2638 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 32s 327ms/step - loss: 0.2469 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 32s 318ms/step - loss: 0.2342 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 33s 337ms/step - loss: 0.2121 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 33s 331ms/step - loss: 0.2034 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 34s 348ms/step - loss: 0.1910 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 33s 333ms/step - loss: 0.1846 - lr: 0.0010
Epoch 11/30
100/100 [===================

# Usage

In [39]:
# берем рандомного юзера и айтем
rand_uid = np.random.choice(list(users_ohe_df.index))
rand_iid = np.random.choice(list(items_ohe_df.index))
k_recos = 10

In [40]:
# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1), np.array(user_interaction_vec).reshape(1, -1)])
# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
ED(user_vec, item_vec)

1/1 [==============================] - 0s 38ms/step


array([[2.9532614]], dtype=float32)

## Готовим на прод

Подготовим лист популярного

In [41]:
name_popular = "../models/pickle_data/popular.pickle"
loaded_popular = pickle.load(open(name_popular, "rb"))
data_for_predict = Dataset.construct(interactions.df)
max_k = len(kion_data["items"]["item_id"].unique())
sample_popular_user = data_for_predict.user_id_map.external_ids[0]
popular_list = list(
    loaded_popular.recommend(dataset=data_for_predict, users=[0], k=max_k, filter_viewed=False)["item_id"]
)

Остальные штучки для предсказаний

In [42]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)
# список юзеров в dssm
users_dssm = list(user_id_to_uid.keys())
# users features
users_meta_feats = users_ohe_df.drop(["user_id"], axis=1)

216/216 [==============================] - 0s 798us/step


Рядом лежит класс dssmmodel.py, который мы заиспользуем

In [43]:
dssm_model = DSSMModel(
    items_vecs=items_vecs,
    users_dssm=users_dssm,
    users_meta_feats=users_meta_feats,
    user_id_to_uid=user_id_to_uid,
    interactions_vec=interactions_vec,
    u2v=u2v,
    iid_to_item_id=iid_to_item_id,
    popular_list=popular_list,
)

### Сохраним модель

In [44]:
name_dssm = "../models/pickle_data/dssm.pickle"
pickle.dump(dssm_model, open(name_dssm, "wb"))

## Проверка работоспособности

In [45]:
name_dssm = "../models/pickle_data/dssm.pickle"
dssm_loaded = pickle.load(open(name_dssm, "rb"))

In [46]:
%%timeit

dssm_loaded.recommend(5324, 10)

77.2 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Должно прокатить, но надо не забыть, что тут не все пользователи, для остальных -- классическое популярное будем возвращзать:)